# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

2012-12-25	Integrating qPLM and biomechanical test data with an anisotropic fiber distribution model and predictions of TGF-1 and IGF-1 regulation of articular cartilage fiber modulus	Biomechanics and Modeling in Mechanobiology	"A continuum mixture model with distinct collagen (COL) and glycosaminoglycan elastic constituents was developed for the solid matrix of immature bovine articular cartilage. A continuous COL fiber volume fraction distribution function and a true COL fiber elastic modulus ([Formula: see text] were used. Quantitative polarized light microscopy (qPLM) methods were developed to account for the relatively high cell density of immature articular cartilage and used with a novel algorithm that constructs a 3D distribution function from 2D qPLM data. For specimens untreated and cultured in vitro, most model parameters were specified from qPLM analysis and biochemical assay results; consequently, [Formula: see text] was predicted using an optimization to measured mechanic

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,category,highlight
0,2018-10-22,Controlling dispersion during single-cell poly...,Analytical Chemistry,New tools for measuring protein expression in ...,"Pan Q, Yamauchi KA, and Herr AE. Controlling D...",single-cell-dispersion,http://academicpages.github.io/files/paper1.pdf,microfluidics,False
1,2017-03-23,Profiling protein expression in circulating tu...,Nature Communications,Circulating tumour cells (CTCs) are rare tumou...,Sinkala E. et al. Profiling protein expression...,single-ctc-western-blotting,http://academicpages.github.io/files/paper2.pdf,microfluidics,False
2,2017-02-13,Subcellular western blotting of single cells,Microsystems and Nanoengineering,Although immunoassays are the de-facto standar...,"Yamauchi KA, Herr AE. Subcellular Western Blot...",subcellular-western-blotting,http://academicpages.github.io/files/paper3.pdf,microfluidics,True
3,2016-09-06,Detection of isoforms differing by a single ch...,Angewandte Chemie,To measure protein isoforms in individual mamm...,"Tentori AM*, Yamauchi KA* and Herr AE. Detecti...",single-cell-ief,http://academicpages.github.io/files/paper3.pdf,microfluidics,True
4,2016-07-28,Single cell-resolution western blotting,Nature Protocols,This protocol describes how to perform western...,"Kang CC, Yamauchi KA*, Vlassakis J*, Sinkala E...",single-cell-western-blotting,http://academicpages.github.io/files/paper3.pdf,microfluidics,True
5,2018-01-31,Arrayed isoelectric focusing using photopatter...,Electrophoresis,Isoelectric focusing (IEF) is a powerful separ...,"Yamauchi KA*, Tentori AM*, and Herr AE. Arraye...",photopatterned-ief-arrays,http://academicpages.github.io/files/paper3.pdf,microfab,True
6,2017-08-21,Fabrication of an open microfluidic device for...,Analytical Chemistry,Given the wide adoption of polydimethylsiloxan...,"Abdel-Sayed P, Yamauchi KA, Gerver RE, Herr AE...",pdms-polyacrylamide-fabrication,http://academicpages.github.io/files/paper3.pdf,microfab,True
7,2012-12-25,Integrating qPLM and biomechanical test data w...,Biomechanics and Modeling in Mechanobiology,A continuum mixture model with distinct collag...,"Stender ME, Raub CB, Yamauchi KA, Shirazi R, V...",mixed-cartilage-model,http://academicpages.github.io/files/paper3.pdf,biomech,True


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + str(item.url_slug) + ".md"
    html_filename = str(item.pub_date) + "-" + str(item.url_slug)
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + str(item.title) + '"\n'
    
    md += """collection: """ + item.category
    
    md += """\npermalink: """ + """/""" + item.category + """/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    if item.highlight:
        md += "\nhighlight: " + "true"
        
    else:
        md += "\nhighlight: " + "false"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\n<b>Recommended citation:</b> " + item.citation
    
    md_filename = os.path.basename(md_filename)
    
    if item.category == "biomech":
        directory = "../_res_biomech/"
        
    elif item.category == "microfluidics":
        directory = "../_microfluidics/"
    
    elif item.category == "microfab":
        directory = "../_microfab/"
    
    elif item.category == "in_situ":
        directory = "../_in_situ/"
    
    else:
        directory = "../_publications/"
       
    with open(directory + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md 2015-10-01-paper-title-number-3.md
2010-10-01-paper-title-number-2.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).